In [1]:
import time
start_time = time.time()

import pandas as pd

In [2]:
MMSTA_path = "../Data/Raw/MMSTA UPLOAD DHN BENITA LOWDASH.XLSX"

In [3]:
df_mmsta = pd.read_excel(MMSTA_path)
df_mmsta

,Level,Plant,Material Type,Material,Material Description,Bill of material,Plant-sp.matl status,Plant-sp.matl status.1,BOM status,BOM status.1,...,MRP profile,MRP profile.1,Lot size,Minimum Lot Size,Rounding value,Spare part indicator,Item Category,Component unit,Component quantity,Status
0,0,MA13,YFG,F000292917,240105971V,6760379.0,2,NaN,1,0,...,A111,NaN,EX,0,0,NaN,L,NaN,0,NaN
1,*1,MA13,YSFG,S008548980,Super Group 020 See drawing,6367154.0,2,NaN,1,0,...,A210,NaN,YX,0,25,NaN,L,PC,1,NaN
2,**2,MA13,YSFG,S008548890,Twisted Wire 26 ZZ,6367066.0,2,NaN,1,0,...,A210,NaN,YX,0,25,NaN,L,PC,1,NaN
3,***3,MA13,YSFG,S008548501,Circuit 1040 COFLRYB 0.35 L/W,6366678.0,92,NaN,92,0,...,A221,NaN,YX,0,25,M,L,PC,1,NaN
4,****4,MA13,YDM,FW406994,LT Single Wire COFLRYB 0.35 L/W,NaN,92,NaN,0,0,...,A305,NaN,ZK,0,14000,NaN,L,MM,1595,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117747,*1,MA13,YDM,430211570,PVC/Vinyl Tape 15mm Y,NaN,2,NaN,0,0,...,A305,NaN,ZK,0,3960,NaN,L,MM,332,NaN
117748,*1,MA13,YDM,430AM4530,Textile/Cloth Tape 45mm B,NaN,2,NaN,0,0,...,A305,NaN,ZK,0,540,NaN,L,MM,4163,NaN
117749,*1,MA13,YDM,430FF1930,PVC/Vinyl Tape 19mm B,NaN,2,NaN,0,0,...,A305,NaN,ZK,0,3168,NaN,L,MM,84914,NaN
117750,*1,MA13,YDM,430H31930,Textile/Cloth Tape 19mm B,NaN,2,NaN,0,0,...,A305,NaN,ZK,0,3600,NaN,L,MM,26693,NaN


In [4]:
df_mmsta.columns

Index(['Level', 'Plant', 'Material Type', 'Material', 'Material Description',
       'Bill of material', 'Plant-sp.matl status', 'Plant-sp.matl status.1',
       'BOM status', 'BOM status.1', 'Change Number', 'Changed on',
       'MRP Controller', 'MRP Controller.1', 'MRP profile', 'MRP profile.1',
       'Lot size', 'Minimum Lot Size', 'Rounding value',
       'Spare part indicator', 'Item Category', 'Component unit',
       'Component quantity', 'Status'],
      dtype='object')

In [5]:
df_mmsta = df_mmsta[['Level', 'Material Type' , 'Material', 'Material Description', 'Component quantity']]

### Add Columns 'PN','SN1','DES1', 'SN2','DES2', 'SN3','DES3'

In [6]:
def add_columns(df):
    global cols_to_add 
    cols_to_add = ['PN','SN1','DES1', 'SN2','DES2', 'SN3','DES3']
    col_index = df.columns.get_loc('Material Description') + 1
    for i, col in enumerate(cols_to_add):
        df.insert(col_index + i, col, '')
    return df
df_mmsta = add_columns(df_mmsta)
df_mmsta

,Level,Material Type,Material,Material Description,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity
0,0,YFG,F000292917,240105971V,,,,,,,,0
1,*1,YSFG,S008548980,Super Group 020 See drawing,,,,,,,,1
2,**2,YSFG,S008548890,Twisted Wire 26 ZZ,,,,,,,,1
3,***3,YSFG,S008548501,Circuit 1040 COFLRYB 0.35 L/W,,,,,,,,1
4,****4,YDM,FW406994,LT Single Wire COFLRYB 0.35 L/W,,,,,,,,1595
...,...,...,...,...,...,...,...,...,...,...,...,...
117747,*1,YDM,430211570,PVC/Vinyl Tape 15mm Y,,,,,,,,332
117748,*1,YDM,430AM4530,Textile/Cloth Tape 45mm B,,,,,,,,4163
117749,*1,YDM,430FF1930,PVC/Vinyl Tape 19mm B,,,,,,,,84914
117750,*1,YDM,430H31930,Textile/Cloth Tape 19mm B,,,,,,,,26693


### Fill PN(Part Number)

In [7]:
def Fill(level : str):
    for idx in range(df_mmsta.shape[0]):
        if df_mmsta.loc[:,'Level'][idx] == level:
            # PN_arr.append(df_mmsta.loc[idx]['Material Description'])
            df_mmsta.loc[idx,'PN'] = df_mmsta.loc[idx]['Material Description']
            idx_last_l = idx
        else :
            df_mmsta.loc[idx,'PN'] = df_mmsta.loc[idx_last_l]['Material Description']
            # PN_arr.append(df_mmsta.loc[idx_last_l]['Material Description'])
    return df_mmsta

In [8]:
df_mmsta = Fill('0')

### Delete YFG And YDM (column = 'Material Type')

In [9]:
def delete_YFG_YDM():
    return df_mmsta[~df_mmsta['Material Type'].isin(['YFG', 'YDM'])]
df_mmsta = delete_YFG_YDM()

### Processing Material Description column (del Cut Tube , Gaft...)

In [10]:
def del_vals(mat_desc : str):
    vals_to_del = ['ctgf', 'gaft', 'cut tube']
    for v in vals_to_del:
        if v in mat_desc.lower():
            return False
    return True

proccessed_log = df_mmsta['Material Description'].apply(del_vals)

df_mmsta = df_mmsta[proccessed_log].reset_index(drop=True)

In [11]:
df_mmsta = df_mmsta.reset_index()

In [12]:
def Fill_with_col(level : str, cols,df):
    idx_last_l = None
    for idx in range(df.shape[0]):
        if df.loc[idx,'Level'] == level:
            df.loc[idx,cols[0]] = df.loc[idx,'Material']
            df.loc[idx,cols[1]] = df.loc[idx,'Material Description']
            idx_last_l = idx
           
            
        else :
            if idx_last_l is not None:
                df.loc[idx,cols[0]] = df.loc[idx_last_l,'Material']
                df.loc[idx,cols[1]] = df.loc[idx_last_l,'Material Description']
                
            else :
                df.loc[idx,cols[0]] = '(blank)'
                df.loc[idx,cols[1]] = '(blank)'
    return df

df_mmsta = Fill_with_col('*1', ['SN1','DES1'], df_mmsta)

df_mmsta = Fill_with_col('**2', ['SN2','DES2'], df_mmsta)

df_mmsta = Fill_with_col('***3', ['SN3','DES3'], df_mmsta)

In [13]:
# import xlwings as xw

# wb = xw.Book()
# sheet_mmsta = wb.sheets['Feuil1']
# sheet_mmsta.range("A1").value = [df_mmsta.columns.tolist()] + df_mmsta.values.tolist()


In [14]:
if 'index' in df_mmsta.columns:
    df_mmsta = df_mmsta.drop(columns='index')
df_mmsta = df_mmsta[['Level' ,'PN','SN1','DES1','SN2','DES2','SN3','DES3', 'Component quantity']]

df_mmsta


,Level,PN,SN1,DES1,SN2,DES2,SN3,DES3,Component quantity
0,*1,240105971V,S008548980,Super Group 020 See drawing,(blank),(blank),(blank),(blank),1
1,**2,240105971V,S008548980,Super Group 020 See drawing,S008548890,Twisted Wire 26 ZZ,(blank),(blank),1
2,***3,240105971V,S008548980,Super Group 020 See drawing,S008548890,Twisted Wire 26 ZZ,S008548501,Circuit 1040 COFLRYB 0.35 L/W,1
3,***3,240105971V,S008548980,Super Group 020 See drawing,S008548890,Twisted Wire 26 ZZ,S008548500,Circuit 1039 COFLRYB 0.35 BR/W,1
4,**2,240105971V,S008548980,Super Group 020 See drawing,S008548889,Twisted Wire 28 ZZ,S008548500,Circuit 1039 COFLRYB 0.35 BR/W,1
...,...,...,...,...,...,...,...,...,...
31570,*1,240100020V,S008536327,Textile Sleeve 10mm B @ 00170,S008548648,Circuit 1294 COFLRYB 0.35 P/W,S008548490,Circuit 1377 COFLRYB 0.35 DL/W,1
31571,*1,240100020V,S008548743,Circuit 0267 COFLRYB 0.35 BR/W,S008548648,Circuit 1294 COFLRYB 0.35 P/W,S008548490,Circuit 1377 COFLRYB 0.35 DL/W,1
31572,*1,240100020V,S008548744,Circuit 0268 COFLRYB 0.35 L/W,S008548648,Circuit 1294 COFLRYB 0.35 P/W,S008548490,Circuit 1377 COFLRYB 0.35 DL/W,1
31573,*1,240100020V,S008548755,Circuit 0308 COFLRYB 0.5 L,S008548648,Circuit 1294 COFLRYB 0.35 P/W,S008548490,Circuit 1377 COFLRYB 0.35 DL/W,1


In [15]:
# df_mmsta = df_mmsta.replace(to_replace='None', value=' ')
# df_mmsta

In [16]:
df_mmsta_pivot = pd.pivot_table(df_mmsta, values = 'Component quantity', index = ['Level','SN1','DES1','SN2' , 'DES2','SN3', 'DES3'], 
                                columns = 'PN', aggfunc = 'size', fill_value = ' ')
# df_mmsta_pivot = df_mmsta_pivot.reset_index()

df_mmsta_pivot

PN                                                                                                                               240100020V  \
Level SN1        DES1                        SN2        DES2                           SN3        DES3                                        
***3  S008548979 Super Group 017 See drawing S008548885 Twisted Wire 64   ZZ           S008548490 Circuit 1377 COFLRYB 0.35 DL/W          1   
                                                                                       S008548491 Circuit 1378 COFLRYB 0.35 G/W           1   
                                             S008548886 Twisted Wire 62   ZZ           S008548492 Circuit 1373 COFLRYB 0.35 DL/W          1   
                                                                                       S008548493 Circuit 1374 COFLRYB 0.35 G/W           1   
                                             S008548887 Twisted Wire 45   ZZ           S008548494 Circuit 1280 COFLRYB 0.35 G/W           1   
...                                                                                                                                     ...   
*1    S008920507 Super Group 025 See drawing S008548658 Circuit 0131 COFLRYB 0.35 L/W  S008548474 Circuit 1085 COFLRYB 0.35 GY/W              
                                             S008548870 Twisted Wire 63   ZZ           S008548443 Circuit 1375 COFLRYB 0.35 BE/W              
                                             S008548881 Twisted Wire 12   ZZ           S008548482 Circuit 1033 COFLRYB 0.35 P/W               
      S008920509 Super Group 010 See drawing S008548688 Circuit 0036 COFLRYB 0.35 GY/W S008548443 Circuit 1375 COFLRYB 0.35 BE/W              
                                             S008548870 Twisted Wire 63   ZZ           S008548443 Circuit 1375 COFLRYB 0.35 BE/W              

PN                                                                                                                               240100114V  \
Level SN1        DES1                        SN2        DES2                           SN3        DES3                                        
***3  S008548979 Super Group 017 See drawing S008548885 Twisted Wire 64   ZZ           S008548490 Circuit 1377 COFLRYB 0.35 DL/W              
                                                                                       S008548491 Circuit 1378 COFLRYB 0.35 G/W               
                                             S008548886 Twisted Wire 62   ZZ           S008548492 Circuit 1373 COFLRYB 0.35 DL/W              
                                                                                       S008548493 Circuit 1374 COFLRYB 0.35 G/W               
                                             S008548887 Twisted Wire 45   ZZ           S008548494 Circuit 1280 COFLRYB 0.35 G/W               
...                                                                                                                                     ...   
*1    S008920507 Super Group 025 See drawing S008548658 Circuit 0131 COFLRYB 0.35 L/W  S008548474 Circuit 1085 COFLRYB 0.35 GY/W              
                                             S008548870 Twisted Wire 63   ZZ           S008548443 Circuit 1375 COFLRYB 0.35 BE/W              
                                             S008548881 Twisted Wire 12   ZZ           S008548482 Circuit 1033 COFLRYB 0.35 P/W               
      S008920509 Super Group 010 See drawing S008548688 Circuit 0036 COFLRYB 0.35 GY/W S008548443 Circuit 1375 COFLRYB 0.35 BE/W              
                                             S008548870 Twisted Wire 63   ZZ           S008548443 Circuit 1375 COFLRYB 0.35 BE/W              

PN                                                                                                                               240100126V  \
Level SN1        DES1                        SN2        DES2                           SN3        DES3                                   

In [17]:
import xlwings as xw

wb = xw.Book()
sheet_mmsta = wb.sheets['Feuil1']
sheet_mmsta.name = "MMSTA"
sheet_mmsta.range("A1").value = df_mmsta_pivot #[df_mmsta.columns.tolist()] + df_mmsta.values.tolist()


In [18]:
df_mmsta_pivot = df_mmsta_pivot.reset_index()

In [19]:
df_mmsta_pivot.loc[df_mmsta_pivot['Level'] == '*1', ['SN2', 'DES2', 'SN3', 'DES3']] = ' '
df_mmsta_pivot.loc[df_mmsta_pivot['Level'] == '**2', ['SN3', 'DES3']] = ' '
df_mmsta_pivot

PN,Level,SN1,DES1,SN2,DES2,SN3,DES3,240100020V,240100114V,240100126V,...,240108529V,240109008V,240109070V,240109175V,240109282V,240109301V,240109523V,240109667V,240109876V,240109949V
0,***3,S008548979,Super Group 017 See drawing,S008548885,Twisted Wire 64 ZZ,S008548490,Circuit 1377 COFLRYB 0.35 DL/W,1,,,...,,,,,1,,1,1,,
1,***3,S008548979,Super Group 017 See drawing,S008548885,Twisted Wire 64 ZZ,S008548491,Circuit 1378 COFLRYB 0.35 G/W,1,,,...,,,,,1,,1,1,,
2,***3,S008548979,Super Group 017 See drawing,S008548886,Twisted Wire 62 ZZ,S008548492,Circuit 1373 COFLRYB 0.35 DL/W,1,,,...,,,,,1,,1,1,,
3,***3,S008548979,Super Group 017 See drawing,S008548886,Twisted Wire 62 ZZ,S008548493,Circuit 1374 COFLRYB 0.35 G/W,1,,,...,,,,,1,,1,1,,
4,***3,S008548979,Super Group 017 See drawing,S008548887,Twisted Wire 45 ZZ,S008548494,Circuit 1280 COFLRYB 0.35 G/W,1,,,...,,,,,1,,1,1,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15913,*1,S008920507,Super Group 025 See drawing,,,,,,,,...,,,,1,,,,,,
15914,*1,S008920507,Super Group 025 See drawing,,,,,,,,...,,,,,,,,,,
15915,*1,S008920507,Super Group 025 See drawing,,,,,,,,...,,,1,,,,,,,
15916,*1,S008920509,Super Group 010 See drawing,,,,,,,,...,,,,,,,,,,


In [20]:
Sep_sheet = wb.sheets.add(after=sheet_mmsta)
Sep_sheet.name = "Sep Sheet"
Sep_sheet.range("A1").value = [df_mmsta_pivot.columns.tolist()] + df_mmsta_pivot.values.tolist()

In [21]:
df_mmsta_pivot[df_mmsta_pivot.columns[7:]] = df_mmsta_pivot[df_mmsta_pivot.columns[7:]].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

In [ ]:
df_fil_simple = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'circuit' in x.lower())]
columns_to_agg = list(df_fil_simple.columns[7:])

agg_dict = {col: 'sum' for col in columns_to_agg}

df_fil_simple = df_fil_simple.groupby(['Level','SN1']).agg({
    'DES1': 'first',
    'SN2': 'first',
    'DES2': 'first',
    'SN3': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()

df_fil_simple.replace(to_replace=" " , value="(Blank)")
df_fil_simple.replace(to_replace= 0 , value=" ")

Fil_simple_sheet = wb.sheets.add(after=Sep_sheet)
Fil_simple_sheet.name = "Fil Simple"
Fil_simple_sheet.range("A1").value = [df_fil_simple.columns.tolist()] + df_fil_simple.values.tolist()

In [23]:
df_double = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'double' in x.lower())]
df_double = df_double.groupby(['Level','SN1']).agg({
    'DES1': 'first',
    'SN2': 'first',
    'DES2': 'first',
    'SN3': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()

double_sheet = wb.sheets.add(after=Fil_simple_sheet)
double_sheet.name = "Double"
double_sheet.range("A1").value = [df_double.columns.tolist()] + df_double.values.tolist()

In [29]:
df_twisted = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'twisted' in x.lower())]
df_twisted = df_twisted.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
twisted_sheet = wb.sheets.add(after=double_sheet)
twisted_sheet.name = "Twisted"
twisted_sheet.range("A1").value = [df_twisted.columns.tolist()] + df_twisted.values.tolist()

In [30]:
df_joint = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'joint' in x.lower())]
df_joint = df_joint.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
joint_sheet = wb.sheets.add(after=twisted_sheet)
joint_sheet.name = "joint"
joint_sheet.range("A1").value = [df_joint.columns.tolist()] + df_joint.values.tolist()

In [31]:
df_super_grp = df_mmsta_pivot [df_mmsta_pivot['DES1'].apply(lambda x : 'super group' in x.lower())]
df_super_grp = df_super_grp.groupby(['Level','SN1','SN2','SN3']).agg({
    'DES1': 'first',
    'DES2': 'first',
    'DES3': 'first',
    **agg_dict
    
}).reset_index()
superGrp_sheet = wb.sheets.add(after=joint_sheet)
superGrp_sheet.name = "Super group"
superGrp_sheet.range("A1").value = [df_super_grp.columns.tolist()] + df_super_grp.values.tolist()

In [32]:
end_time = time.time()

execution_time = end_time - start_time
print(f"Temps d'exécution : {execution_time / 60} minutes")

Temps d'exécution : 10.083380516370138 minutes


In [39]:
df_joint[['SN1','SN2']].value_counts()

SN1         SN2       
S008548902                1
            S008548524    1
            S008548525    1
            S008548526    1
S008905175                1
                         ..
S008905460                1
            S008548589    1
            S008905026    1
            S008905027    1
            S008905248    1
Name: count, Length: 186, dtype: int64

In [25]:
import pandas as pd


data = {
    'level': ['A', 'B', 'C', 'C', 'C', 'A', 'A', 'C', 'B', 'B'],
    'levelB': ['xx', 'D', 'Blank', 'Blank', ' ', 'Blank', 'Blank', ' ', ' ', ' '],
    'Column1': [1 , 1, 0 , 1,1 , 1, 0 , 1,0,0],
    'Column2': [1 , 0, 0 , 0,0 , 1, 1 , 1,0,0],
    'Column3': [1 , 1, 0 , 1,1 , 1, 0 , 1,0,0],
    'Column4': [1 , 1, 0 , 0,0 , 0, 0 , 0,0,0],
    'Column5': [0 , 0, 0 , 0,0 , 1, 0 , 1,0,0]
    
}

df_new = pd.DataFrame(data)
df_new

,level,levelB,Column1,Column2,Column3,Column4,Column5
0,A,xx,1,1,1,1,0
1,B,D,1,0,1,1,0
2,C,Blank,0,0,0,0,0
3,C,Blank,1,0,1,0,0
4,C,,1,0,1,0,0
5,A,Blank,1,1,1,0,1
6,A,Blank,0,1,0,0,0
7,C,,1,1,1,0,1
8,B,,0,0,0,0,0
9,B,,0,0,0,0,0


In [28]:
df_grouped = df_new.groupby(['level','levelB']).agg({
    'Column1': 'sum',
    'Column2': 'sum',
    'Column3': 'sum',
    'Column4': 'sum',
    'Column5': 'sum',
    
}).reset_index()
df_grouped

,level,levelB,Column1,Column2,Column3,Column4,Column5
0,A,Blank,1,2,1,0,1
1,A,xx,1,1,1,1,0
2,B,,0,0,0,0,0
3,B,D,1,0,1,1,0
4,C,,2,1,2,0,1
5,C,Blank,1,0,1,0,0
